In [ ]:
%pip install opencv-python mediapipe==0.10.7
%pip install numpy 

In [ ]:
import mediapipe as mp
import cv2
import numpy as np

In [ ]:
mp_drawing=mp.solutions.drawing_utils
mp_pose=mp.solutions.pose

testing mediapipe and camera feed


In [ ]:
cap = cv2. VideoCapture (0)

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, image = cap. read ()
        # Recolor Feed
        image = cv2. cvtColor (image, cv2. COLOR_BGR2RGB)
        image. flags.writeable = False
        
        results = pose. process (image)
        
        image.flags. writeable = True
        image = cv2. cvtColor (image, cv2. COLOR_RGB2BGR)
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose. POSE_CONNECTIONS
                                    , mp_drawing. DrawingSpec(color=(0,255,0), thickness=2, circle_radius=4), 
                                    mp_drawing. DrawingSpec(color=(0,0,255), thickness=2, circle_radius=2)
                                
        )
        cv2. imshow( 'Raw Webcam Feed', image)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()
        
        


Creating a video to train model

In [ ]:
cap = cv2. VideoCapture (0)
height = cap.get (cv2. CAP_PROP_FRAME_HEIGHT)
width = cap. get (cv2.CAP_PROP_FRAME_WIDTH)
fps = cap.get (cv2. CAP_PROP_FPS)
videoWriter = cv2. VideoWriter('press.avi', cv2.VideoWriter_fourcc('P','I', 'M','1'), fps, (int(width), int(height)))
while cap. isOpened():
    ret, frame = cap.read()
    try:
        cv2. imshow( 'Press', frame)
        videoWriter.write (frame)
    except Exception as e:
        break
    if cv2.waitKey (1) & 0xFF == ord( 'q'):
        break
cap.release
videoWriter.release ()
cv2. destroyAllWindows ( )

Testing if the video captured can be detected by mediapipe

In [ ]:
cap = cv2. VideoCapture ('press.avi')
with mp_pose. Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap. read ()
        if not ret:
            break
        # Recolor Feed
        image = cv2.cvtColor (frame, cv2. COLOR_BGR2RGB)
        image.flags.writeable = False
        # Make Detections
        results = pose.process (image)
        # Recolor image back to BGR for rendering
        image. flags.writeable= True
        image = cv2.cvtColor (image, cv2. COLOR_RGB2BGR)
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose. POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec (color=(0,255,0), thickness=2,circle_radius=4),
                                  mp_drawing.DrawingSpec (color=(0,0,255), thickness=2,circle_radius=4)
                                
                                )
        k = cv2. waitKey(1)
        cv2. imshow( 'Raw Webcam Feed', image)
        if cv2.waitKey(1)& 0xFF == ord('q'):
            break
cap. release
cv2. destroyAllWindows ()

In [ ]:
print(results.pose_landmarks)

annotating the dataset

In [ ]:
import csv 
import os
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
landmarks = ['class']
for val in range(1, 33+1):
    landmarks += ['x{}'.format(val), 'y{}'. format(val), 'z{}'. format(val), 'v{}'. format(val)]

In [ ]:
with open('data/coords.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv. QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [ ]:
def export_landmark(results, action) :
    print(results.pose_landmarks)
    try:
        keypoints = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten().tolist()
        keypoints.insert(0, action)
        with open( 'data/coords.csv', mode='a' , newline='') as f:
            csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            csv_writer.writerow(keypoints)
    except Exception as e:
        pass

main code for annotation

In [ ]:
cap = cv2. VideoCapture ('videos/press.avi')
with mp_pose. Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap. read ()
        if not ret:
            break
        # Recolor Feed
        image = cv2.cvtColor (frame, cv2. COLOR_BGR2RGB)
        image.flags.writeable = False
        # Make Detections
        results = pose.process (image)
        # Recolor image back to BGR for rendering
        image. flags.writeable= True
        image = cv2.cvtColor (image, cv2. COLOR_RGB2BGR)
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose. POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec (color=(0,255,0), thickness=2,circle_radius=4),
                                  mp_drawing.DrawingSpec (color=(0,0,255), thickness=2,circle_radius=4)
                                
                                )
        k = cv2. waitKey(1)
        if k == 117:
            export_landmark(results, 'up')
        if k == 100:
            export_landmark(results, 'down')
        cv2. imshow( 'Raw Webcam Feed', image)
        if cv2.waitKey(1)& 0xFF == ord('q'):
            break
cap. release
cv2. destroyAllWindows ()

Traning custom model


In [ ]:
import pandas as pd 
from sklearn.model_selection import train_test_split

In [ ]:
df=pd.read_csv('data/coords.csv')


In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df[df['class']=='up']

In [ ]:
X = df.drop( 'class', axis=1) # features
y = df['class'] # target value


In [ ]:
X_train, X_test, Y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

In [ ]:
y_test

Train Machine Learning Classification Model

In [ ]:
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression, RidgeClassifier 
from sklearn. ensemble import RandomForestClassifier, GradientBoostingClassifier

In [ ]:
pipelines = {
'Ir' :make_pipeline(StandardScaler(), LogisticRegression()),
'rc' :make_pipeline (StandardScaler(), RidgeClassifier()),
'rf' :make_pipeline(StandardScaler(), RandomForestClassifier()),
'gb' :make_pipeline (StandardScaler(), GradientBoostingClassifier()),
}

In [ ]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, Y_train)
    fit_models[algo] = model

In [ ]:
fit_models

In [ ]:
fit_models ['rc'].predict(X_test)

serialize using pickel

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score # Accuracy metrics 
import pickle

In [ ]:
for algo, model in fit_models. items():
    yhat = model. predict(X_test)
    print(algo, accuracy_score(y_test.values, yhat),
            precision_score(y_test.values,yhat, average="binary", pos_label="up"), 
            recall_score(y_test.values, yhat, average="binary", pos_label="up"))

In [ ]:
yhat=fit_models['rf'].predict(X_test)

In [ ]:
with open( 'deadlift.pkl', 'wb') as f:
    pickle.dump (fit_models['rf'], f)

making predictions

In [1]:
import mediapipe as mp
import cv2
import numpy as np
import pandas as pd 
import pickle

mp_drawing=mp.solutions.drawing_utils
mp_pose=mp.solutions.pose


In [4]:
with open('../models/deadlift.pkl', 'rb') as f:
    model=pickle.load(f)

In [5]:
cap = cv2. VideoCapture (0)
counter=0
current_stage=''
landmarks = ['class']
for val in range(1, 33+1):
    landmarks += ['x{}'.format(val), 'y{}'. format(val), 'z{}'. format(val),'v{}'.format(val)]

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, image = cap. read ()
# Recolor Feed
        image = cv2.cvtColor (image, cv2.COLOR_BGR2RGB)
        image. flags.writeable = False
        
        results = pose. process (image)
        
        image.flags. writeable = True
        image = cv2.cvtColor (image, cv2.COLOR_RGB2BGR)
        
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose. POSE_CONNECTIONS
                                    , mp_drawing. DrawingSpec(color=(0,255,0), thickness=2, circle_radius=4), 
                                    mp_drawing. DrawingSpec(color=(0,0,255), thickness=2, circle_radius=2)
                                
        )
        try:
            row = np.array([[res.x, res.y, res.z, res.visibility]for res in results.pose_landmarks.landmark]).flatten().tolist()  # Show the Bounding Box around the Body
            X = pd.DataFrame([row], columns=landmarks[1:])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            print(body_language_class, body_language_prob)
            
            
            if body_language_class == 'down' and body_language_prob[body_language_prob.argmax()]>=0.55: 
                current_stage = 'down'
            elif current_stage =='down' and body_language_class =='up' and body_language_prob[body_language_prob.argmax()]>0.55:
                current_stage="up"
                counter =counter+1
                print(current_stage)

            cv2. rectangle(image, (0,0), (250, 60), (245, 117, 16), - 1)
# Display CLass
            cv2. putText (image, 'CLASS',
                    (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class.split(' ')[0], (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2,cv2.LINE_AA)
            
            cv2. putText(image, 'PROB',(15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2. putText(image, str(round(body_language_prob[np.argmax(body_language_prob)], 2)), (10,40), cv2. FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2. LINE_AA)
 
 
            cv2. putText (image, 'COUNT', (180,12), cv2. FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2. putText (image,str(counter), (175,40), cv2. FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2,cv2.LINE_AA)
            
        
        except Exception as e:
            print(e)
            pass
        
        
        cv2. imshow( 'Raw Webcam Feed', image)
        if cv2.waitKey (10) & 0xFF == ord('q'):
            break
cap. release ( )
cv2. destroyAllWindows ()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


'NoneType' object has no attribute 'landmark'


2024-05-02 23:39:03.103 Python[73905:3199812] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute

: 